In [5]:
import pandas
import numpy as np
df = pandas.read_csv("Saturday, April 02, 2016")
df.columns = ['C/A','UNIT','SCP','STATION','LINENAME','DIVISION','DATE','TIME','DESC','IN','OUT']
df = df[['UNIT', 'SCP', 'DATE', 'TIME', "IN", "OUT"]]
df.head(5)

,UNIT,SCP,DATE,TIME,IN,OUT
0,R051,02-00-00,03/26/2016,00:00:00,5595746,1893277
1,R051,02-00-00,03/26/2016,04:00:00,5595746,1893282
2,R051,02-00-00,03/26/2016,08:00:00,5595746,1893282
3,R051,02-00-00,03/26/2016,12:00:00,5595746,1893282
4,R051,02-00-00,03/26/2016,16:00:00,5595746,1893282


In [6]:
geocode = pandas.read_csv('geocoded.csv', header=None)
geocode = geocode.drop_duplicates(0)
geocode = geocode[[0,5,6]]
geocode.columns = ['UNIT', 'LAT', 'LON']
geocode_mapping = {row.values[0]:[row[1], row[2]] for index,row in geocode.iterrows()}

In [7]:
def map(unit):
    try:
        return geocode_mapping[unit]
    except:
        return np.nan

df['COORD'] = df['UNIT'].apply(map)

In [8]:
df.head(20)

,UNIT,SCP,DATE,TIME,IN,OUT,COORD
0,R051,02-00-00,03/26/2016,00:00:00,5595746,1893277,"[40.762796, -73.967686]"
1,R051,02-00-00,03/26/2016,04:00:00,5595746,1893282,"[40.762796, -73.967686]"
2,R051,02-00-00,03/26/2016,08:00:00,5595746,1893282,"[40.762796, -73.967686]"
3,R051,02-00-00,03/26/2016,12:00:00,5595746,1893282,"[40.762796, -73.967686]"
4,R051,02-00-00,03/26/2016,16:00:00,5595746,1893282,"[40.762796, -73.967686]"
5,R051,02-00-00,03/26/2016,20:00:00,5595746,1893282,"[40.762796, -73.967686]"
6,R051,02-00-00,03/27/2016,00:00:00,5595746,1893282,"[40.762796, -73.967686]"
7,R051,02-00-00,03/27/2016,04:00:00,5595746,1893282,"[40.762796, -73.967686]"
8,R051,02-00-00,03/27/2016,08:00:00,5595746,1893282,"[40.762796, -73.967686]"
9,R051,02-00-00,03/27/2016,12:00:00,5595746,1893282,"[40.762796, -73.967686]"


In [9]:
import datetime
df["DATETIME"] = df["DATE"]+ ' ' + df["TIME"]

In [10]:
df["DATETIME"] = pandas.to_datetime(df["DATETIME"])

In [11]:
day = []
for i in range(0,df["DATETIME"].shape[0]):
    if df["DATETIME"][i].isoweekday() in range(1,6):
        day.append("Weekday")
    else:
        day.append("Weekend")

In [12]:
df["DAY"] = day

In [13]:
x = [1,2,3,4,5,6,5000,5001,5002,5003,5004,5005]
df.loc[x]

,UNIT,SCP,DATE,TIME,IN,OUT,COORD,DATETIME,DAY
1,R051,02-00-00,03/26/2016,04:00:00,5595746,1893282,"[40.762796, -73.967686]",2016-03-26 04:00:00,Weekend
2,R051,02-00-00,03/26/2016,08:00:00,5595746,1893282,"[40.762796, -73.967686]",2016-03-26 08:00:00,Weekend
3,R051,02-00-00,03/26/2016,12:00:00,5595746,1893282,"[40.762796, -73.967686]",2016-03-26 12:00:00,Weekend
4,R051,02-00-00,03/26/2016,16:00:00,5595746,1893282,"[40.762796, -73.967686]",2016-03-26 16:00:00,Weekend
5,R051,02-00-00,03/26/2016,20:00:00,5595746,1893282,"[40.762796, -73.967686]",2016-03-26 20:00:00,Weekend
6,R051,02-00-00,03/27/2016,00:00:00,5595746,1893282,"[40.762796, -73.967686]",2016-03-27 00:00:00,Weekend
5000,R170,03-00-00,04/01/2016,09:00:00,1288655,9042769,"[40.734836, -73.990688]",2016-04-01 09:00:00,Weekday
5001,R170,03-00-00,04/01/2016,13:00:00,1288729,9044085,"[40.734836, -73.990688]",2016-04-01 13:00:00,Weekday
5002,R170,03-00-00,04/01/2016,17:00:00,1288970,9045642,"[40.734836, -73.990688]",2016-04-01 17:00:00,Weekday
5003,R170,03-00-00,04/01/2016,21:00:00,1289235,9047852,"[40.734836, -73.990688]",2016-04-01 21:00:00,Weekday


In [39]:
masterDF = pandas.DataFrame(columns=['UNIT', 'IN', 'OUT', 'COORDS'])
for keyA, groupA in df.groupby(['UNIT']):
    stationDF = pandas.DataFrame(columns=df.columns)
    for key, group in groupA.groupby("SCP"):
        #print "KEY:", key
        #print "group:", group

        diff = group["IN"].iloc[1:].values - group["IN"].iloc[:-1].values
        diff = np.concatenate((np.array([0]), diff))
        group['IN'] = diff
        group['IN'].iloc[0] = np.nan

        diff = group["OUT"].iloc[1:].values - group["OUT"].iloc[:-1].values
        diff = np.concatenate((np.array([0]), diff))
        group['OUT'] = diff
        group['OUT'].iloc[0] = np.nan

        #group['FLUX'] = (group['OUT'] / group['IN'] - 1) * (group['IN'] + group['OUT'])
        #group['DIFF'] =  group['OUT'] - group['IN']
    
        stationDF = stationDF.append(group)
        
    weekdays = newDF[newDF.DAY == 'Weekday']
    def classify_time(time):
        if 5 <= time.hour <= 9:
            return "Morning"
        elif 17 < time.hour < 22:
            return "Evening"
        else:
            return None
    weekdays["M_E"] = weekdays['DATETIME'].apply(classify_time)
    mornings = weekdays[weekdays.M_E == 'Morning']
    masterDF.loc[len(masterDF)] = (stationDF['UNIT'].iloc[0], stationDF['IN'].sum(), 
                                   stationDF['OUT'].sum(), stationDF['COORD'].iloc[0])


    break


/Library/Python/2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/ipykernel/__main__.py:16: Sett

In [40]:
masterDF

,UNIT,IN,OUT,COORDS
0,R001,187722,186009,"[40.703082, -74.012983]"


In [35]:

import time, calendar, datetime, numpy
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import urllib, os
# data downloaded from the form at
# http://coastwatch.pfeg.noaa.gov/erddap/tabledap/apdrcArgoAll.html
filename, headers = urllib.urlretrieve('http://coastwatch.pfeg.noaa.gov/erddap/tabledap/apdrcArgoAll.nc?longitude,latitude,time&longitude>=0&longitude<=360&latitude>=-90&latitude<=90&time>=2010-01-01&time<=2010-01-08&distinct()')
dset = Dataset(filename)
lats = dset.variables['latitude'][:]
lons = dset.variables['longitude'][:]
time = dset.variables['time']
times = time[:]
t1 = times.min(); t2 = times.max()
date1 = num2date(t1, units=time.units)
date2 = num2date(t2, units=time.units)
dset.close()
os.remove(filename)
# draw map with markers for float locations
m = Basemap(projection='hammer',lon_0=180)
x, y = m(lons,lats)
m.drawmapboundary(fill_color='#99ffff')
m.fillcontinents(color='#cc9966',lake_color='#99ffff')
m.scatter(x,y,3,marker='o',color='k')
plt.title('Locations of %s ARGO floats active between %s and %s' %\
        (len(lats),date1,date2),fontsize=12)
plt.show()

ImportError: No module named basemap